## Convolutional Neural Networks

Convolution layers in Neural Networks apply filters to images to help the network decide what features are important for learning. They take much longer to train than a dense network.

#### Some examples of convolutional filters and how they augment images:
![edge](images/edge2.png)

### Convolution filters make feature maps of images which are incorperated into the neural network's learning.
![file_name](http://ufldl.stanford.edu/tutorial/images/Convolution_schematic.gif)

### Features are then pooled
![file](http://ufldl.stanford.edu/tutorial/images/Cnn_layer.png)

### Max pooling works like this:
![file_name](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

## Filter visualization

Thes images are generated with a deep convolutional network called Imagenet VGG16.  It is a very deep neural network that is trained with 150,000 images in 1000 classes.  The weights for this network are open source and available for everyone to use.

More information about Imagent is here: http://image-net.org/

### Features for an image after two convolution layers
![file_name](https://raw.githubusercontent.com/raghakot/keras-vis/master/images/conv_vis/block1_conv2_filters.jpg?raw=true)

### Features for an image after ten convolution layers
![file_name](https://raw.githubusercontent.com/raghakot/keras-vis/master/images/conv_vis/block4_conv3_filters.jpg?raw=true)

In [12]:
from __future__ import print_function
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

#### To force your environment to run in tensorflow use the following command in terminal

KERAS_BACKEND=tensorflow; python -c "from keras import backend"

In [3]:
batch_size = 128
nb_classes = 5
nb_epoch = 5
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

now = datetime.datetime.now

# allows code to be compatable with tensorflow and theano
if K.image_dim_ordering() == 'th':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

The following function defines the model and also does some image augmentation.  

Because our images are simple and are organized for us by Keras we don't have to do very much data handling.  If we were building a neural net from our own images Keras' image processing library would come in handy. 

For future reference here is the link to the image processing library!
https://keras.io/preprocessing/image/

In [4]:
def train_model(model, train, test, nb_classes):
    # image processing
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [5]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create a dataset with digits below 5
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

In [6]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

In [7]:
# create complete model
model = Sequential(feature_layers + classification_layers)

In [8]:
# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 69s - loss: 0.2150 - acc: 0.9350 - val_loss: 0.0401 - val_acc: 0.9868